In [ ]:
import pandas as pd
df = pd.read_table('Twitter-Absolute-Sigma-500.data',sep=',',header=None)

In [ ]:
df.head()

In [ ]:
df[77].unique()

In [ ]:
df.info()

In [ ]:
df[77].value_counts()

In [ ]:
df.shape

In [ ]:
df_1 = df.sample(frac=0.1,random_state=5)

In [ ]:
df_1

In [ ]:
X = df_1.iloc[:,:77]
y = df_1[77]

In [ ]:
X

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.dummy import DummyClassifier
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot as plt
from sklearn import linear_model, datasets
import mglearn
from sklearn import svm, grid_search
from sklearn.tree import DecisionTreeClassifier

In [ ]:
X_train_org, X_test_org, y_train, y_test = train_test_split(X,y,random_state=5)
sc = StandardScaler()
X_train = sc.fit_transform(X_train_org)
X_test = sc.transform(X_test_org)

print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

In [ ]:
dummy_majority = DummyClassifier(strategy='most_frequent').fit(X_train, y_train)

pred_most_frequent = dummy_majority.predict(X_test)

print("Unique predicted labels: {}".format(np.unique(pred_most_frequent)))
print("Test score: {:.2f}".format(dummy_majority.score(X_test, y_test)))

In [ ]:
knn = KNeighborsClassifier()
#param_grid = {'n_neighbors':[1, 5, 10, 15, 20]}
param_grid = {'n_neighbors':[1, 5, 10, 15, 20, 25, 30, 35, 40, 45]}
scoring = {'AUC': 'roc_auc', 'Recall': 'recall', 'Precision': 'precision'}

grid_knn = GridSearchCV(knn, param_grid=param_grid, cv = 5, scoring=scoring, refit='AUC')
grid_knn.fit(X_train, y_train)
grid_knn.score(X_train, y_train)

In [ ]:
file = grid_knn.cv_results_
file

In [ ]:
grid_knn.score(X_test, y_test)

In [ ]:
grid_knn.best_estimator_

In [ ]:
y_knn_predict = grid_knn.predict(X_test)
y_knn_train_predict = grid_knn.predict(X_train)

In [ ]:
print('Train roc_auc_score: %.2f'%roc_auc_score(y_knn_train_predict, y_train))
print('Test roc_auc_score: %.2f '%roc_auc_score(y_knn_predict, y_test))

In [ ]:
Fit = ['Model', 'Best Parameter', 'ROC_AUC Train Score', 'ROC_AUC Test Score']
knn_report = [['KNN', grid_knn.best_params_, roc_auc_score(y_knn_train_predict, y_train), roc_auc_score(y_knn_predict, y_test)]]
pd.DataFrame(knn_report, columns=Fit)

In [ ]:
### Logistic Regression

In [ ]:
logistic = linear_model.LogisticRegression()

scoring = {'AUC': 'roc_auc', 'Recall': 'recall', 'Precision': 'precision'}
penalty = ['l1', 'l2']
c_range = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
hyperparameters = dict(C=c_range, penalty=penalty)

grid_lr = GridSearchCV(logistic, cv=5, verbose=0, scoring=scoring, param_grid=hyperparameters, refit='AUC')
grid_lr.fit(X_train, y_train)
grid_lr.score(X_train, y_train)

In [ ]:
grid_lr.score(X_test, y_test)

In [ ]:
grid_lr.best_params_

In [ ]:
grid_lr.best_estimator_

In [ ]:
y_predict = grid_lr.predict(X_test)
y_train_predict = grid_lr.predict(X_train)

In [ ]:
print('Train roc_auc_score: %.2f'%roc_auc_score(y_train_predict, y_train))
print('Test roc_auc_score: %.2f '%roc_auc_score(y_predict, y_test))

In [ ]:
result = grid_lr.cv_results_
pd.DataFrame(result)

In [ ]:
logistic_report = [['Logistic Regression', grid_lr.best_params_, roc_auc_score(y_train_predict, y_train), roc_auc_score(y_predict, y_test)]]
pd.DataFrame(logistic_report, columns=Fit)

In [ ]:
### Linear SVC

In [ ]:
scoring = {'AUC': 'roc_auc', 'Recall': 'recall', 'Precision': 'precision'}
Cs = [0.001, 0.01, 0.1, 1, 10]
#gammas = [0.001, 0.01, 0.1, 1]
param_grid = {'C': Cs}
grid_linear_svc = GridSearchCV(svm.SVC(), param_grid, cv=5, refit='AUC', scoring=scoring)
grid_linear_svc.fit(X_train, y_train)
grid_linear_svc.score(X_train, y_train)

In [ ]:
grid_linear_svc.score(X_test, y_test)

In [ ]:
grid_linear_svc.best_params_

In [ ]:
grid_linear_svc.best_estimator_

In [ ]:
y_linear_svc_predict = grid_linear_svc.predict(X_test)
y_linear_svc_train_predict = grid_linear_svc.predict(X_train)

print('Train roc_auc_score: %.2f'%roc_auc_score(y_linear_svc_train_predict, y_train))
print('Test roc_auc_score: %.2f '%roc_auc_score(y_linear_svc_predict, y_test))

In [ ]:
result = grid_linear_svc.cv_results_
result

In [ ]:
linear_svc_report = [['Linear SVC', grid_linear_svc.best_params_, roc_auc_score(y_linear_svc_train_predict, y_train), roc_auc_score(y_linear_svc_predict, y_test)]]
pd.DataFrame(linear_svc_report, columns=Fit)

In [ ]:
### Kernelized SVM

In [ ]:
scoring = {'AUC': 'roc_auc', 'Recall': 'recall', 'Precision': 'precision'}
Cs = [0.001, 0.01, 0.1, 1, 10]
gammas = [0.001, 0.01, 0.1, 1]
param_grid = {'C': Cs, 'gamma' : gammas}
grid_svc = GridSearchCV(svm.SVC(kernel='rbf'), param_grid, cv=5, refit='AUC', scoring=scoring)
grid_svc.fit(X_train, y_train)
grid_svc.score(X_train, y_train) 

In [ ]:
grid_svc.score(X_test, y_test)

In [ ]:
grid_svc.best_params_

In [ ]:
grid_svc.best_estimator_

In [ ]:
y_svc_predict = grid_svc.predict(X_test)
y_svc_train_predict = grid_svc.predict(X_train)

print('Train roc_auc_score: %.2f'%roc_auc_score(y_svc_train_predict, y_train))
print('Test roc_auc_score: %.2f '%roc_auc_score(y_svc_predict, y_test))

In [ ]:
result = grid_svc.cv_results_
pd.DataFrame(result)

In [ ]:
kernalised_svc_report = [['Kernalised SVC', grid_svc.best_params_, roc_auc_score(y_svc_train_predict, y_train), roc_auc_score(y_svc_predict, y_test)]
pd.DataFrame(kernalised_svc_report, columns=Fit)

In [ ]:
### Decision Tree

In [ ]:
scoring = {'AUC': 'roc_auc', 'Recall': 'recall', 'Precision': 'precision'}
param_grid = {'max_depth':[1,2,3,4,5]}
dtree_grid = GridSearchCV(DecisionTreeClassifier(random_state=5),param_grid=param_grid,cv=5,return_train_score=True, refit='AUC', scoring=scoring)
dtree_grid.fit(X_train,y_train)
dtree_grid.score(X_train,y_train)

In [ ]:
dtree_grid.score(X_test,y_test)

In [ ]:
pd.DataFrame(dtree_grid.cv_results_)

In [ ]:
dtree_grid.best_params_

In [ ]:
dtree_grid.best_estimator_

In [ ]:
y_dtree_predict = dtree_grid.predict(X_test)
y_dtree_train_predict = dtree_grid.predict(X_train)

print('Train roc_auc_score: %.2f'%roc_auc_score(y_dtree_train_predict, y_train))
print('Test roc_auc_score: %.2f '%roc_auc_score(y_dtree_predict, y_test))

In [ ]:
dtree=DecisionTreeClassifier(max_depth=4)
dtree.fit(X_train,y_train)

In [ ]:
%matplotlib inline
def plot_feature_importances(model):
    n_features = X_train.data.shape[1]
    plt.barh(range(n_features), model.feature_importances_, align='center')
    plt.yticks(np.arange(0,n_features,3))
    plt.xlabel("Feature importance")
    plt.ylabel("Feature")
    plt.ylim(-1, n_features)
plt.figure(figsize=(10,5))
plot_feature_importances(dtree)

In [ ]:
x=X_train[:,[33,34]]
dtree1=DecisionTreeClassifier(max_depth=3)
dtree1.fit(x,y_train)
mglearn.plots.plot_2d_separator(dtree1, x, fill=True, eps=0.5, alpha=.4)
mglearn.discrete_scatter(x[:, 0], x[:, 1], y_train)

In [ ]:
dtree_report = [['Decision Tree', dtree_grid.best_params_, roc_auc_score(y_dtree_train_predict, y_train), roc_auc_score(y_dtree_predict, y_test)]]
pd.DataFrame(dtree_report, columns=Fit)

In [ ]:
Report = pd.DataFrame(knn_report+logistic_report+linear_svc_report+kernalised_svc_report+dtree_report, columns=Titles, index=range(1,6))
Report

In [ ]:
plt.plot(Report.iloc[:,0],Report.iloc[:,2],label='ROC Train Score')
plt.plot(Report.iloc[:,0],Report.iloc[:,3],label='ROC Test Score')
plt.legend()

In [ ]:
### From the plot, the logistic regression is the model which gives us the best ROC test scores. 
### The parameters {'C': 0.1, 'penalty': 'l1'} found on the data set.

In [ ]:
all_X = df.iloc[:,:77]
all_y = df[77]

In [ ]:
all_X_train_org, all_X_test_org, all_y_train, all_y_test = train_test_split(all_X,all_y,random_state=5)
sc = StandardScaler()
all_X_train = sc.fit_transform(all_X_train_org)
all_X_test = sc.transform(all_X_test_org)

all_logistic = linear_model.LogisticRegression(C=0.1,penalty='l1')
all_logistic.fit(all_X_train,all_y_train)

In [ ]:
all_logistic.score(all_X_train,all_y_train)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
log = linear_model.LogisticRegression()
fold = KFold(n_splits=5)

In [ ]:
cv_train_score = cross_val_score(log, all_X_train,all_y_train, cv=fold)
cv_test_score = cross_val_score(log, all_X_test,all_y_test, cv=fold)

In [ ]:
print('Cross validation train score is : {0}\nCross validation test score is : {1}'.format(round(cv_train_score.mean(),5), round(cv_test_score.mean(),5)))